In [16]:
import pandas as pd
from common import Data

In [17]:
df = Data.get_nwsl_goals(season="2023")
df["opponent"] = df.apply(
    lambda x: x["home"] if x["home"] != x["team"] else x["away"], axis=1
)
df

,season,date,home,away,player,team,assist,period,minute,second,x,y,opponent
0,2023,2023-03-25,NC,KC,Mille Gejl Jensen,NC,Narumi Miura,1,22,24,70.4,63.8,KC
1,2023,2023-03-25,SD,CHI,Yuki Ogimi Nagasato,CHI,NaN,1,17,41,97.6,47.4,SD
2,2023,2023-03-25,SD,CHI,Amirah Ali,SD,NaN,1,21,34,98.3,48.4,CHI
3,2023,2023-03-25,SD,CHI,Jaedyn Shaw,SD,NaN,1,31,7,81.6,50.4,CHI
4,2023,2023-03-25,SD,CHI,Mallory Pugh Swanson,CHI,NaN,1,48,18,88.5,50.0,SD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,2023,2023-10-15,ORL,HOU,Marta,ORL,NaN,2,86,5,88.5,50.0,HOU
336,2023,2023-10-15,SD,LOU,Jaedyn Shaw,SD,Alex Morgan,1,25,46,92.6,53.6,LOU
337,2023,2023-10-15,SD,LOU,Alex Morgan,SD,Madison Pogarch,2,46,19,84.3,62.7,LOU
338,2023,2023-10-15,WAS,NC,Tyler Lussi,NC,Manaka Matsukubo,1,24,43,95.0,54.3,WAS


In [18]:
cols = {
    "0-15": (1, 0, 15),
    "15-30": (1, 15, 30),
    "30-45+": (1, 30, 90),
    "45-60": (2, 45, 60),
    "60-75": (2, 60, 75), 
    "75-90+": (2, 75, 135), 
}

for col in cols:
    half, lower, upper = cols[col]
    df[col] = (df["period"] == half) & (df["minute"] >= lower) & (df["minute"] < upper)
    
df = df.drop(columns=["period", "minute", "x", "y", "second"])
    
df

,season,date,home,away,player,team,assist,opponent,0-15,15-30,30-45+,45-60,60-75,75-90+
0,2023,2023-03-25,NC,KC,Mille Gejl Jensen,NC,Narumi Miura,KC,False,True,False,False,False,False
1,2023,2023-03-25,SD,CHI,Yuki Ogimi Nagasato,CHI,NaN,SD,False,True,False,False,False,False
2,2023,2023-03-25,SD,CHI,Amirah Ali,SD,NaN,CHI,False,True,False,False,False,False
3,2023,2023-03-25,SD,CHI,Jaedyn Shaw,SD,NaN,CHI,False,False,True,False,False,False
4,2023,2023-03-25,SD,CHI,Mallory Pugh Swanson,CHI,NaN,SD,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,2023,2023-10-15,ORL,HOU,Marta,ORL,NaN,HOU,False,False,False,False,False,True
336,2023,2023-10-15,SD,LOU,Jaedyn Shaw,SD,Alex Morgan,LOU,False,True,False,False,False,False
337,2023,2023-10-15,SD,LOU,Alex Morgan,SD,Madison Pogarch,LOU,False,False,False,True,False,False
338,2023,2023-10-15,WAS,NC,Tyler Lussi,NC,Manaka Matsukubo,WAS,False,True,False,False,False,False


In [19]:
scored = df.groupby("team").sum()
scored.to_csv("data/goals_scored_2023.csv")
scored

,0-15,15-30,30-45+,45-60,60-75,75-90+
team,,,,,,
CHI,1,5,6,6,8,1
HOU,2,2,5,4,3,1
KC,5,7,4,5,7,4
LA,3,2,8,3,6,7
LOU,3,4,4,5,3,4
NC,3,5,8,6,3,3
NJY,5,2,3,4,7,5
ORL,3,10,3,4,5,3
POR,8,9,6,5,8,8


In [22]:
scored["1"] = scored["0-15"] + scored["15-30"] + scored["30-45+"]
scored["2"] = scored["45-60"] + scored["60-75"] + scored["75-90+"]
scored["ratio"] = scored["1"]/ (scored["1"] + scored["2"])
scored.sort_values(["ratio"])

,0-15,15-30,30-45+,45-60,60-75,75-90+,1,2,ratio
team,,,,,,,,,
RGN,2,2,4,7,5,9,8,21,0.275862
SD,3,4,4,10,4,6,11,20,0.354839
NJY,5,2,3,4,7,5,10,16,0.384615
WAS,3,4,4,5,4,6,11,15,0.423077
CHI,1,5,6,6,8,1,12,15,0.444444
LA,3,2,8,3,6,7,13,16,0.448276
LOU,3,4,4,5,3,4,11,12,0.478261
KC,5,7,4,5,7,4,16,16,0.500000
POR,8,9,6,5,8,8,23,21,0.522727


In [23]:
conceded = df.groupby("opponent").sum()
conceded.to_csv("data/goals_conceded_2023.csv")
conceded

,0-15,15-30,30-45+,45-60,60-75,75-90+
opponent,,,,,,
CHI,9,7,10,10,8,7
HOU,2,0,2,6,4,3
KC,5,5,8,6,5,5
LA,2,3,6,6,9,7
LOU,4,5,4,4,5,4
NC,4,3,3,3,5,5
NJY,1,4,4,6,6,2
ORL,3,6,5,3,5,5
POR,2,6,4,8,5,5
